<a href="https://colab.research.google.com/github/DeependraParichha1004/NLP/blob/main/BertTokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q -U "tensorflow-text==2.8.*"

     |████████████████████████████████| 4.9 MB 5.0 MB/s 


In [ ]:
! pip install -q tensorflow_datasets

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import tensorflow_text as text
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert_vocab
import time
import pandas as pd
import numpy as np
import pathlib
import re

In [ ]:
examples,metadata=tfds.load('ted_hrlr_translate/pt_to_en',with_info=True,as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteEP9R9Z/ted_hrlr_translate-train.tfrecord


...

  0%|          | 0/1193 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0.incompleteEP9R9Z/ted_hrlr_translate-test.tfrecord


  0%|          | 0/1803 [00:00<?, ? examples/s]

Dataset ted_hrlr_translate downloaded and prepared to /root/tensorflow_datasets/ted_hrlr_translate/pt_to_en/1.0.0. Subsequent calls will reuse this data.


In [ ]:
train_examples,validation_examples=examples['train'],examples['validation']

In [ ]:
type(train_examples)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

## English Dataset

In [ ]:
import numpy
for pt,en in train_examples.batch(2).take(1):
    for en1 in en.numpy():
      print(en1.decode('utf-8'))

and when you improve searchability , you actually take away the one advantage of print , which is serendipity .
but what if it were active ?


In [ ]:
train_en=train_examples.map(lambda pt,en:en)

In [ ]:
train_en

<MapDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

## Hindi Dataset

In [ ]:
import pandas as pd
data=pd.read_csv('/content/Hindi_English_Truncated_Corpus.csv')
hin=data['hindi_sentence'].iloc[:].values
hin=tf.data.Dataset.from_tensor_slices(hin)
type(hin)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [ ]:
data_=data['hindi_sentence']

In [ ]:
data_

0         राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह कर...
1         मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहू...
2          यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3            हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4               इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
                                ...                        
127602    आर्ट डेको शैली के निर्माण मैरीन ड्राइव और ओवल ...
127603                      और अपने गालों में डाल लेते हैं।
127604    जहां तक गंधक के अन्य उत्पादों का प्रश्न है , द...
127605    Zरचना-प्रकिया को उसने एक पहेली में यों बांधा है .
127606    हाल ही में उन्हें सरकारी ठेका मिला है करीब सौ ...
Name: hindi_sentence, Length: 127607, dtype: object

In [ ]:
data_[10]

'कृपया यह सुनिश्चित कर लें कि आप सही फॉर्म का प्रयोग कर रहें हैं .'

In [ ]:
def hin_dataset(ds):
  return (
      ds
      .cache()
      .prefetch(tf.data.AUTOTUNE)
  )

In [ ]:
hindi_examples=hin_dataset(hin)
hindi_examples

<PrefetchDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [ ]:
for ex in hindi_examples.batch(2).take(1):
  for ex1 in ex.numpy():
    print(ex1.decode('utf-8'))

राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है .
मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,


## English and Hindi Vocab

#### Hindi Vocab

In [ ]:
bert_tokenizer_params=dict(lower_case=True)
reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]

In [ ]:
bert_vocab_args = dict(
    # The target vocabulary size
    vocab_size = 8000,
    # Reserved tokens that must be included in the vocabulary
    reserved_tokens=reserved_tokens,
    # Arguments for `text.BertTokenizer`
    bert_tokenizer_params=bert_tokenizer_params,
    # Arguments for `wordpiece_vocab.wordpiece_tokenizer_learner_lib.learn`
    learn_params={},
)

In [ ]:
def write_vocab(file_path,en_vocab):
    with open(file_path,'w',encoding='utf-8') as f:
        for words in en_vocab:
            print(words,file=f)

In [ ]:
hindi_examples

<PrefetchDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [ ]:
%%time
hin_vocab=bert_vocab.bert_vocab_from_dataset(
hindi_examples.batch(1000).prefetch(2),
**bert_vocab_args)

CPU times: user 2min 39s, sys: 844 ms, total: 2min 40s
Wall time: 2min 45s


In [ ]:
write_vocab('hin_vocab.txt', hin_vocab)

#### English Vocab

In [ ]:
%%time
en_vocab=bert_vocab.bert_vocab_from_dataset(
train_en.batch(1000).prefetch(2),
**bert_vocab_args)

CPU times: user 1min 41s, sys: 3.07 s, total: 1min 44s
Wall time: 1min 40s


In [ ]:
write_vocab('en_vocab.txt', en_vocab)

## English and Hindi Tokenizer

### English Tokenizer

In [ ]:
import tensorflow_text as text
en_tokenizer=text.BertTokenizer('en_vocab.txt',**bert_tokenizer_params)


In [ ]:
for en_examples in train_en.batch(3).take(1):
    print(en_examples.numpy())

[b'and when you improve searchability , you actually take away the one advantage of print , which is serendipity .'
 b'but what if it were active ?' b"but they did n't test for curiosity ."]


In [ ]:
sequences_eng=en_tokenizer.tokenize(en_examples)
sequences_eng

<tf.RaggedTensor [[[72], [117], [79], [1259], [1491, 2362], [13], [79], [150], [184], [311],
  [71], [103], [2308], [74], [2679], [13], [148], [80],
  [55, 4840, 1434, 2423, 540], [15]]                                       ,
 [[87], [90], [107], [76], [129], [1852], [30]],
 [[87], [83], [149], [50], [9], [56], [664], [85], [2512], [15]]]>

In [ ]:
sequences_eng=sequences_eng.merge_dims(-2,-1)#merge_dims->(word,word-piece)
sequences_eng

<tf.RaggedTensor [[72, 117, 79, 1259, 1491, 2362, 13, 79, 150, 184, 311, 71, 103, 2308, 74,
  2679, 13, 148, 80, 55, 4840, 1434, 2423, 540, 15]                       ,
 [87, 90, 107, 76, 129, 1852, 30],
 [87, 83, 149, 50, 9, 56, 664, 85, 2512, 15]]>

Words like "searchability" and "serendipity" <br>have been decomposed into "search ##ability" and "s ##ere ##nd<br>##ip ##ity" using tf.gather.

In [ ]:
tokens_eng=tf.gather(en_vocab,sequences_eng)
tokens_eng

<tf.RaggedTensor [[b'and', b'when', b'you', b'improve', b'search', b'##ability', b',',
  b'you', b'actually', b'take', b'away', b'the', b'one', b'advantage',
  b'of', b'print', b',', b'which', b'is', b's', b'##ere', b'##nd', b'##ip',
  b'##ity', b'.']                                                          ,
 [b'but', b'what', b'if', b'it', b'were', b'active', b'?'],
 [b'but', b'they', b'did', b'n', b"'", b't', b'test', b'for', b'curiosity',
  b'.']                                                                    ]>

In [ ]:
sent_en=en_tokenizer.detokenize(sequences_eng)
tf.strings.reduce_join(sent_en,separator=' ',axis=-1)

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'and when you improve searchability , you actually take away the one advantage of print , which is serendipity .',
       b'but what if it were active ?',
       b"but they did n ' t test for curiosity ."], dtype=object)>

In [ ]:
sent_en

<tf.RaggedTensor [[b'and', b'when', b'you', b'improve', b'searchability', b',', b'you',
  b'actually', b'take', b'away', b'the', b'one', b'advantage', b'of',
  b'print', b',', b'which', b'is', b'serendipity', b'.']              ,
 [b'but', b'what', b'if', b'it', b'were', b'active', b'?'],
 [b'but', b'they', b'did', b'n', b"'", b't', b'test', b'for', b'curiosity',
  b'.']                                                                    ]>

### Hindi Tokenizer

In [ ]:
hin_tokenizer=text.BertTokenizer('hin_vocab.txt',**bert_tokenizer_params)

In [ ]:
for hn1 in hindi_examples.batch(3).take(1):
  print(hn1.numpy())

[b'\xe0\xa4\xb0\xe0\xa4\xbe\xe0\xa4\x9c\xe0\xa4\xa8\xe0\xa5\x80\xe0\xa4\xa4\xe0\xa4\xbf\xe0\xa4\x9c\xe0\xa5\x8d\xe0\xa4\x9e\xe0\xa5\x8b\xe0\xa4\x82 \xe0\xa4\x95\xe0\xa5\x87 \xe0\xa4\xaa\xe0\xa4\xbe\xe0\xa4\xb8 \xe0\xa4\x9c\xe0\xa5\x8b \xe0\xa4\x95\xe0\xa4\xbe\xe0\xa4\xb0\xe0\xa5\x8d\xe0\xa4\xaf \xe0\xa4\x95\xe0\xa4\xb0\xe0\xa4\xa8\xe0\xa4\xbe \xe0\xa4\x9a\xe0\xa4\xbe\xe0\xa4\xb9\xe0\xa4\xbf\xe0\xa4\x8f, \xe0\xa4\xb5\xe0\xa4\xb9 \xe0\xa4\x95\xe0\xa4\xb0\xe0\xa4\xa8\xe0\xa5\x87 \xe0\xa4\x95\xe0\xa4\xbf \xe0\xa4\x85\xe0\xa4\xa8\xe0\xa5\x81\xe0\xa4\xae\xe0\xa4\xa4\xe0\xa4\xbf \xe0\xa4\xa8\xe0\xa4\xb9\xe0\xa5\x80\xe0\xa4\x82 \xe0\xa4\xb9\xe0\xa5\x88 .'
 b'\xe0\xa4\xae\xe0\xa4\x88 \xe0\xa4\x86\xe0\xa4\xaa\xe0\xa4\x95\xe0\xa5\x8b \xe0\xa4\x90\xe0\xa4\xb8\xe0\xa5\x87 \xe0\xa4\xb9\xe0\xa5\x80 \xe0\xa4\x8f\xe0\xa4\x95 \xe0\xa4\xac\xe0\xa4\x9a\xe0\xa5\x8d\xe0\xa4\x9a\xe0\xa5\x87 \xe0\xa4\x95\xe0\xa5\x87 \xe0\xa4\xac\xe0\xa4\xbe\xe0\xa4\xb0\xe0\xa5\x87 \xe0\xa4\xae\xe0\xa5\x87\xe0\xa4\x82 \xe0\xa4

In [ ]:
for hn1 in hindi_examples.batch(3).take(1):
  for hin1 in hn1.numpy():
    print(hin1.decode('utf-8'))

राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है .
मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,
यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।


In [ ]:
sequences_hin=hin_tokenizer.tokenize(hn1)
sequences_hin


<tf.RaggedTensor [[[1186, 6339], [105], [308], [205], [370], [260], [286], [15], [219],
  [209], [194], [1049], [197], [138], [17]]                           ,
 [[1481], [361], [334], [202], [192], [535], [105], [237], [129], [2263],
  [5997], [15]]                                                          ,
 [[196], [441], [227], [129], [1849], [441], [137], [254], [138], [150]]]>

In [ ]:
sequences_hin=sequences_hin.merge_dims(-2,-1)
sequences_hin

<tf.RaggedTensor [[1186, 6339, 105, 308, 205, 370, 260, 286, 15, 219, 209, 194, 1049, 197,
  138, 17]                                                               ,
 [1481, 361, 334, 202, 192, 535, 105, 237, 129, 2263, 5997, 15],
 [196, 441, 227, 129, 1849, 441, 137, 254, 138, 150]]>

In [ ]:
detokens=hin_tokenizer.detokenize(sequences_hin)
tf.strings.reduce_join(detokens,separator='',axis=-1)


<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'\xe0\xa4\xb0\xe0\xa4\xbe\xe0\xa4\x9c\xe0\xa4\xa8\xe0\xa5\x80\xe0\xa4\xa4\xe0\xa4\xbf\xe0\xa4\x9c\xe0\xa4\x9e\xe0\xa5\x8b\xe0\xa4\x95\xe0\xa4\xaa\xe0\xa4\xbe\xe0\xa4\xb8\xe0\xa4\x9c\xe0\xa5\x8b\xe0\xa4\x95\xe0\xa4\xbe\xe0\xa4\xb0\xe0\xa4\xaf\xe0\xa4\x95\xe0\xa4\xb0\xe0\xa4\xa8\xe0\xa4\xbe\xe0\xa4\x9a\xe0\xa4\xbe\xe0\xa4\xb9\xe0\xa4\xbf\xe0\xa4\x8f,\xe0\xa4\xb5\xe0\xa4\xb9\xe0\xa4\x95\xe0\xa4\xb0\xe0\xa4\xa8\xe0\xa4\x95\xe0\xa4\xbf\xe0\xa4\x85\xe0\xa4\xa8\xe0\xa4\xae\xe0\xa4\xa4\xe0\xa4\xbf\xe0\xa4\xa8\xe0\xa4\xb9\xe0\xa5\x80\xe0\xa4\xb9.',
       b'\xe0\xa4\xae\xe0\xa4\x88\xe0\xa4\x86\xe0\xa4\xaa\xe0\xa4\x95\xe0\xa5\x8b\xe0\xa4\x90\xe0\xa4\xb8\xe0\xa4\xb9\xe0\xa5\x80\xe0\xa4\x8f\xe0\xa4\x95\xe0\xa4\xac\xe0\xa4\x9a\xe0\xa4\x9a\xe0\xa4\x95\xe0\xa4\xac\xe0\xa4\xbe\xe0\xa4\xb0\xe0\xa4\xae\xe0\xa4\xac\xe0\xa4\xa4\xe0\xa4\xbe\xe0\xa4\xa8\xe0\xa4\xbe\xe0\xa4\x9a\xe0\xa4\xbe\xe0\xa4\xb9\xe0\xa4\x97\xe0\xa5\x80,',
       b'\xe0\xa4\xaf\xe0\xa

In [ ]:
a=tf.strings.reduce_join(detokens,separator=' ',axis=-1)
for elem in a.numpy():
  print(elem.decode('utf-8'))

राजनीतिजञो क पास जो कारय करना चाहिए , वह करन कि अनमति नही ह .
मई आपको ऐस ही एक बचच क बार म बताना चाहगी ,
यह परतिशत भारत म हिनदओ परतिशत स अधिक ह ।


In [ ]:
for row in txt_tokens:
  for elem in row.numpy():
    print(elem.decode('utf-8'),separator='')

राजनीति
##जञो
क
पास
जो
कारय
करना
चाहिए
,
वह
करन
कि
अनमति
नही
ह
.
मई
आपको
ऐस
ही
एक
बचच
क
बार
म
बताना
चाहगी
,
यह
परतिशत
भारत
म
हिनदओ
परतिशत
स
अधिक
ह
।


To re-assemble words from the extracted tokens,

In [ ]:
sent=en_tokenizer.detokenize(sequences_eng)
tf.strings.reduce_join(sent,separator=' ',axis=-1)

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'and when you improve searchability , you actually take away the one advantage of print , which is serendipity .',
       b'but what if it were active ?',
       b"but they did n ' t test for curiosity ."], dtype=object)>

Customization and Export

In [ ]:
START = tf.argmax(tf.constant(reserved_tokens) == "[START]")
END = tf.argmax(tf.constant(reserved_tokens) == "[END]")

def add_start_end(ragged):
  count = ragged.bounding_shape()[0]
  starts = tf.fill([count,1], START)
  ends = tf.fill([count,1], END)
  return tf.concat([starts, ragged, ends], axis=1)

In [ ]:
words = en_tokenizer.detokenize(add_start_end(sequences_eng))
tf.strings.reduce_join(words, separator=' ', axis=-1)

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'[START] and when you improve searchability , you actually take away the one advantage of print , which is serendipity . [END]',
       b'[START] but what if it were active ? [END]',
       b"[START] but they did n ' t test for curiosity . [END]"],
      dtype=object)>

In [ ]:
words

<tf.RaggedTensor [[b'[START]', b'and', b'when', b'you', b'improve', b'searchability', b',',
  b'you', b'actually', b'take', b'away', b'the', b'one', b'advantage',
  b'of', b'print', b',', b'which', b'is', b'serendipity', b'.', b'[END]'] ,
 [b'[START]', b'but', b'what', b'if', b'it', b'were', b'active', b'?',
  b'[END]']                                                           ,
 [b'[START]', b'but', b'they', b'did', b'n', b"'", b't', b'test', b'for',
  b'curiosity', b'.', b'[END]']                                          ]>

In [ ]:
def cleanup_text(reserved_tokens, token_txt):
  # Drop the reserved tokens, except for "[UNK]".
  bad_tokens = [re.escape(tok) for tok in reserved_tokens if tok != "[UNK]"]
  bad_token_re = "|".join(bad_tokens)

  bad_cells = tf.strings.regex_full_match(token_txt, bad_token_re)
  result = tf.ragged.boolean_mask(token_txt, ~bad_cells)

  # Join them into strings.
  result = tf.strings.reduce_join(result, separator=' ', axis=-1)

  return result

In [ ]:
cleanup_text(reserved_tokens, words).numpy()

array([b'and when you improve searchability , you actually take away the one advantage of print , which is serendipity .',
       b'but what if it were active ?',
       b"but they did n ' t test for curiosity ."], dtype=object)

In [ ]:
sequences_eng

<tf.RaggedTensor [[72, 117, 79, 1259, 1491, 2362, 13, 79, 150, 184, 311, 71, 103, 2308, 74,
  2679, 13, 148, 80, 55, 4840, 1434, 2423, 540, 15]                       ,
 [87, 90, 107, 76, 129, 1852, 30],
 [87, 83, 149, 50, 9, 56, 664, 85, 2512, 15]]>

In [ ]:
words=en_tokenizer.detokenize(sequences_eng)
tf.strings.reduce_join(words,separator=' ',axis=-1)

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'and when you improve searchability , you actually take away the one advantage of print , which is serendipity .',
       b'but what if it were active ?',
       b"but they did n ' t test for curiosity ."], dtype=object)>

In [ ]:
sequences_hin

<tf.RaggedTensor [[1186, 6339, 105, 308, 205, 370, 260, 286, 15, 219, 209, 194, 1049, 197,
  138, 17]                                                               ,
 [1481, 361, 334, 202, 192, 535, 105, 237, 129, 2263, 5997, 15],
 [196, 441, 227, 129, 1849, 441, 137, 254, 138, 150]]>

In [ ]:
words_1=hin_tokenizer.detokenize(sequences_hin)
words_1=tf.strings.reduce_join(words_1,separator=' ',axis=-1)
words_1

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'\xe0\xa4\xb0\xe0\xa4\xbe\xe0\xa4\x9c\xe0\xa4\xa8\xe0\xa5\x80\xe0\xa4\xa4\xe0\xa4\xbf\xe0\xa4\x9c\xe0\xa4\x9e\xe0\xa5\x8b \xe0\xa4\x95 \xe0\xa4\xaa\xe0\xa4\xbe\xe0\xa4\xb8 \xe0\xa4\x9c\xe0\xa5\x8b \xe0\xa4\x95\xe0\xa4\xbe\xe0\xa4\xb0\xe0\xa4\xaf \xe0\xa4\x95\xe0\xa4\xb0\xe0\xa4\xa8\xe0\xa4\xbe \xe0\xa4\x9a\xe0\xa4\xbe\xe0\xa4\xb9\xe0\xa4\xbf\xe0\xa4\x8f , \xe0\xa4\xb5\xe0\xa4\xb9 \xe0\xa4\x95\xe0\xa4\xb0\xe0\xa4\xa8 \xe0\xa4\x95\xe0\xa4\xbf \xe0\xa4\x85\xe0\xa4\xa8\xe0\xa4\xae\xe0\xa4\xa4\xe0\xa4\xbf \xe0\xa4\xa8\xe0\xa4\xb9\xe0\xa5\x80 \xe0\xa4\xb9 .',
       b'\xe0\xa4\xae\xe0\xa4\x88 \xe0\xa4\x86\xe0\xa4\xaa\xe0\xa4\x95\xe0\xa5\x8b \xe0\xa4\x90\xe0\xa4\xb8 \xe0\xa4\xb9\xe0\xa5\x80 \xe0\xa4\x8f\xe0\xa4\x95 \xe0\xa4\xac\xe0\xa4\x9a\xe0\xa4\x9a \xe0\xa4\x95 \xe0\xa4\xac\xe0\xa4\xbe\xe0\xa4\xb0 \xe0\xa4\xae \xe0\xa4\xac\xe0\xa4\xa4\xe0\xa4\xbe\xe0\xa4\xa8\xe0\xa4\xbe \xe0\xa4\x9a\xe0\xa4\xbe\xe0\xa4\xb9\xe0\xa4\x97\xe0\xa5\x80 ,',
   

In [ ]:
cleanup_text(reserved_tokens,words_)

In [ ]:
class CustomTokenizer(tf.Module):
  def __init__(self, reserved_tokens, vocab_path):
    self.tokenizer = text.BertTokenizer(vocab_path, lower_case=True)
    self._reserved_tokens = reserved_tokens
    self._vocab_path = tf.saved_model.Asset(vocab_path)

    vocab = pathlib.Path(vocab_path).read_text().splitlines()
    self.vocab = tf.Variable(vocab)

    ## Create the signatures for export:   

    # Include a tokenize signature for a batch of strings. 
    self.tokenize.get_concrete_function(
        tf.TensorSpec(shape=[None], dtype=tf.string))

    # Include `detokenize` and `lookup` signatures for:
    #   * `Tensors` with shapes [tokens] and [batch, tokens]
    #   * `RaggedTensors` with shape [batch, tokens]
    self.detokenize.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.detokenize.get_concrete_function(
          tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    self.lookup.get_concrete_function(
        tf.TensorSpec(shape=[None, None], dtype=tf.int64))
    self.lookup.get_concrete_function(
          tf.RaggedTensorSpec(shape=[None, None], dtype=tf.int64))

    # These `get_*` methods take no arguments
    self.get_vocab_size.get_concrete_function()
    self.get_vocab_path.get_concrete_function()
    self.get_reserved_tokens.get_concrete_function()

  @tf.function
  def tokenize(self, strings):
    enc = self.tokenizer.tokenize(strings)
    # Merge the `word` and `word-piece` axes.
    enc = enc.merge_dims(-2,-1)
    enc = add_start_end(enc)
    return enc

  @tf.function
  def detokenize(self, tokenized):
    words = self.tokenizer.detokenize(tokenized)
    return cleanup_text(self._reserved_tokens, words)

  @tf.function
  def lookup(self, token_ids):
    return tf.gather(self.vocab, token_ids)

  @tf.function
  def get_vocab_size(self):
    return tf.shape(self.vocab)[0]

  @tf.function
  def get_vocab_path(self):
    return self._vocab_path

  @tf.function
  def get_reserved_tokens(self):
    return tf.constant(self._reserved_tokens)

In [ ]:
tokenizers = tf.Module()
tokenizers.en = CustomTokenizer(reserved_tokens, 'en_vocab.txt')
tokenizers.hin = CustomTokenizer(reserved_tokens, 'hin_vocab.txt')

## Save Model using saved_model

In [ ]:
model_name = 'ted_hrlr_translate_hn_en_converter'
tf.saved_model.save(tokenizers, model_name)

INFO:tensorflow:Assets written to: ted_hrlr_translate_hn_en_converter/assets


INFO:tensorflow:Assets written to: ted_hrlr_translate_hn_en_converter/assets


In [ ]:
reloaded_tokenizers = tf.saved_model.load(model_name)
reloaded_tokenizers.en.get_vocab_size().numpy()

7010

In [ ]:
reloaded_tokenizers.hin.get_vocab_size().numpy()

7612

In [ ]:
tokens_eng = reloaded_tokenizers.en.tokenize(['Hello TensorFlow!'])
tokens_eng.numpy()

array([[   2, 4006, 2358,  687, 1192, 2365,    4,    3]])

In [ ]:
detokens_eng=reloaded_tokenizers.en.detokenize(tokens_eng)
detokens_eng

<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'hello tensorflow !'], dtype=object)>

In [ ]:
print(detokens_eng.numpy()[0].decode('utf-8'))

hello tensorflow !


In [ ]:
tokens = reloaded_tokenizers.hin.tokenize(['कृपया यह सुनिश्चित कर लें कि आप सही फॉर्म का प्रयोग कर रहें हैं'])
tokens


<tf.RaggedTensor [[2, 2654, 196, 1789, 200, 132, 194, 222, 587, 5886, 190, 380, 200, 230,
  138, 3]]>

In [ ]:
text_tokens = reloaded_tokenizers.hin.lookup(tokens)
text_tokens

<tf.RaggedTensor [[b'[START]', b'\xe0\xa4\x95\xe0\xa4\xaa\xe0\xa4\xaf\xe0\xa4\xbe',
  b'\xe0\xa4\xaf\xe0\xa4\xb9',
  b'\xe0\xa4\xb8\xe0\xa4\xa8\xe0\xa4\xbf\xe0\xa4\xb6\xe0\xa4\x9a\xe0\xa4\xbf\xe0\xa4\xa4',
  b'\xe0\xa4\x95\xe0\xa4\xb0', b'\xe0\xa4\xb2',
  b'\xe0\xa4\x95\xe0\xa4\xbf', b'\xe0\xa4\x86\xe0\xa4\xaa',
  b'\xe0\xa4\xb8\xe0\xa4\xb9\xe0\xa5\x80',
  b'\xe0\xa4\xab\xe0\xa5\x89\xe0\xa4\xb0\xe0\xa4\xae',
  b'\xe0\xa4\x95\xe0\xa4\xbe',
  b'\xe0\xa4\xaa\xe0\xa4\xb0\xe0\xa4\xaf\xe0\xa5\x8b\xe0\xa4\x97',
  b'\xe0\xa4\x95\xe0\xa4\xb0', b'\xe0\xa4\xb0\xe0\xa4\xb9',
  b'\xe0\xa4\xb9', b'[END]']]>

In [ ]:
round_trip = reloaded_tokenizers.hin.detokenize(tokens)

In [ ]:
round_trip

<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'\xe0\xa4\x95\xe0\xa4\xaa\xe0\xa4\xaf\xe0\xa4\xbe \xe0\xa4\xaf\xe0\xa4\xb9 \xe0\xa4\xb8\xe0\xa4\xa8\xe0\xa4\xbf\xe0\xa4\xb6\xe0\xa4\x9a\xe0\xa4\xbf\xe0\xa4\xa4 \xe0\xa4\x95\xe0\xa4\xb0 \xe0\xa4\xb2 \xe0\xa4\x95\xe0\xa4\xbf \xe0\xa4\x86\xe0\xa4\xaa \xe0\xa4\xb8\xe0\xa4\xb9\xe0\xa5\x80 \xe0\xa4\xab\xe0\xa5\x89\xe0\xa4\xb0\xe0\xa4\xae \xe0\xa4\x95\xe0\xa4\xbe \xe0\xa4\xaa\xe0\xa4\xb0\xe0\xa4\xaf\xe0\xa5\x8b\xe0\xa4\x97 \xe0\xa4\x95\xe0\xa4\xb0 \xe0\xa4\xb0\xe0\xa4\xb9 \xe0\xa4\xb9'],
      dtype=object)>

In [ ]:
type(round_trip)

tensorflow.python.framework.ops.EagerTensor

In [ ]:
round_trip.numpy()[0]

b'\xe0\xa4\x95\xe0\xa4\xaa\xe0\xa4\xaf\xe0\xa4\xbe \xe0\xa4\xaf\xe0\xa4\xb9 \xe0\xa4\xb8\xe0\xa4\xa8\xe0\xa4\xbf\xe0\xa4\xb6\xe0\xa4\x9a\xe0\xa4\xbf\xe0\xa4\xa4 \xe0\xa4\x95\xe0\xa4\xb0 \xe0\xa4\xb2 \xe0\xa4\x95\xe0\xa4\xbf \xe0\xa4\x86\xe0\xa4\xaa \xe0\xa4\xb8\xe0\xa4\xb9\xe0\xa5\x80 \xe0\xa4\xab\xe0\xa5\x89\xe0\xa4\xb0\xe0\xa4\xae \xe0\xa4\x95\xe0\xa4\xbe \xe0\xa4\xaa\xe0\xa4\xb0\xe0\xa4\xaf\xe0\xa5\x8b\xe0\xa4\x97 \xe0\xa4\x95\xe0\xa4\xb0 \xe0\xa4\xb0\xe0\xa4\xb9 \xe0\xa4\xb9'

In [ ]:
model_name_hn_en='ted_hrlr_translate_hn_en_converter'
tokenizers_hn_en=tf.saved_model.load(model_name_hn_en)

In [ ]:
tokens_en1=tokenizers_hn_en.en.tokenize(['Hi This is an Engineer .'])
tokens_en1

<tf.RaggedTensor [[2, 2265, 81, 80, 111, 3334, 15, 3]]>

In [ ]:
detokens_en1=tokenizers_hn_en.en.detokenize(tokens_en1)
detokens_en1

<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'hi this is an engineer .'], dtype=object)>

### Hindi Sentence(Example)

In [ ]:
tokens_hn1=tokenizers_hn_en.hin.tokenize(['और अपने गालों में डाल लेते हैं ।'])
tokens_hn1

<tf.RaggedTensor [[2, 189, 213, 107, 1024, 204, 129, 1160, 818, 138, 150, 3]]>

In [ ]:
tokens_new_hn1=tokens_hn1.merge_dims(-2,-1)
tokens_new_hn1

<tf.Tensor: shape=(12,), dtype=int64, numpy=
array([   2,  189,  213,  107, 1024,  204,  129, 1160,  818,  138,  150,
          3])>

In [ ]:
detokens_hn1=tokenizers_hn_en.hin.detokenize(tokens_hn1)
for element in detokens_hn1.numpy():
  print(element.decode('utf-8'))

और अपन गालो म डाल लत ह ।




>                                  OR



In [ ]:
tokens = reloaded_tokenizers.hin.tokenize(['कृपया यह सुनिश्चित कर लें कि आप सही फॉर्म का प्रयोग कर रहें हैं'])
round_trip = reloaded_tokenizers.hin.detokenize(tokens)


In [ ]:
print(round_trip.numpy()[0].decode('utf-8'))

कपया यह सनिशचित कर ल कि आप सही फॉरम का परयोग कर रह ह


### Portuguese Sentences(Example)

In [ ]:
for element in pt_examples.numpy():
  print(element.decode('utf-8'))

e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade .
mas e se estes fatores fossem ativos ?
mas eles não tinham a curiosidade de me testar .


In [ ]:
tokens_pt=tokenizers.pt.tokenize(pt_examples)
tokens_pt

<tf.RaggedTensor [[2, 44, 115, 6402, 148, 40, 887, 14, 3936, 40, 463, 2715, 94, 2047, 14, 84,
  44, 40, 117, 1328, 2721, 818, 539, 16, 3]                                 ,
 [2, 99, 44, 89, 199, 2836, 1336, 3996, 32, 3],
 [2, 99, 131, 88, 383, 40, 2673, 83, 110, 1972, 16, 3]]>

In [ ]:
detokens_pt=tokenizers.pt.detokenize(tokens_pt)
detokens_pt

<tf.Tensor: shape=(3,), dtype=string, numpy=
array([b'e quando melhoramos a procura , tiramos a unica vantagem da impressao , que e a serendipidade .',
       b'mas e se estes fatores fossem ativos ?',
       b'mas eles nao tinham a curiosidade de me testar .'], dtype=object)>